In [1]:
###############################################################
# File: Ratio_70_75_80.ipynb
# Date: August 2, 2021
# Version: 1.0
# Author: Matthew Varela
# Description: 
#  This program reads in Mesonet data and outputs data related to heat bursts
#
# Copright (c) 2021
# Board of Regents, Univ. of Oklahoma 
# All Rights Reserved.
# Proprietary. Confidential.
###############################################################


import pandas as pd
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import calendar
from math import exp, log, radians, degrees, sin, cos, atan2, pi, tan, acos, asin
import sys
sys.path.append('/home/billston/public_html/python/lib/')
from mesonet_data_server import SiteQuery
from ok_mesonet_libs import *

def mesonet_time_series_data(stids, stime, etime, parms):
    """ Get Mesonet time series data from Data Portal.

    :param stids:  List of Mesonet site IDs.
    :param stime:  Start date/time.
    :param etime:  End date/time.
    :param parms:  List of Mesonet variable IDs.
    :return:       Dataframe of Mesonet data.
    """
    units = {"tair":"cels", "ta9m":"cels", "tapp":"cels", "relh":"prct", "tdew":"cels", "gwetb":"fahr", 
             "wspd":"mps", "ws2m":"mps", "wmax":"mps", "wdir":"degr", "rain":"inch",
             "pres":"mb", "palt":"mb", "srad":"wpsm", "exsr2":"wpsm",
             "fw05":"nada", "fw25":"nada", "fw60":"nada", "fw75":"nada", 
             "mp05":"kpas", "mp25":"kpas", "mp60":"kpas", "mp75":"kpas",
             "wc05":"m3m3", "wc25":"m3m3", "wc60":"m3m3", "wc75":"m3m3",
             "paw10":"mm", "paw40":"mm", "paw80":"mm",
             "faw10":"mm", "faw40":"mm", "faw80":"mm",
             "tb05":"cels", "ts05":"cels", "tb10":"cels", "ts10":"cels", "ts25":"cels", "ts30":"cels", "ts60":"cels"}
    _TIMEFMT = "%Y-%m-%d %H:%M:%S"
    mts_table = pd.DataFrame()
    for stid in (stids):
        query = SiteQuery()
        query.request.update({
            "station": stid,
            "sdate": stime.strftime(query.TIMEFMT),
            "edate": etime.strftime(query.TIMEFMT),
            "interval": 300,
            "variables": [{"id": parm, "units" : units[parm]} for parm in parms]
        })
        query.submit()
        mts_table = mts_table.append(query.table())
    mts_table.drop(["stid","time"], axis=1, inplace=True)
    mts_table.reset_index(inplace=True)
    return mts_table                    

#### Main Program ####    
if __name__ == "__main__":
    print("** Begin Program **")
    
    # Mesonet Data Variables
    gust_max_threshold= 10
    heat_burst_threshold=10
    heat_burst_max_temp_threshold= 25
    tair_threshold= 2.7
    tdew_depr_threshold= 5.4
    dewpt_dpt_change_threshold= 3.0

    #Stations and Parameters
    params = ['tair','tdew', 'wmax']
    mesonet_siteinfo_file = "/mnt/remote/stor3/archive/mesonet/info/siteinfo.xml"
    siteinfo, stids = parse_siteinfo(mesonet_siteinfo_file)
    stids.sort()
    stids = ['ACME','ADAX','ALTU','ALV2','ALVA','ANT2','ANTL','APAC','ARD2','ARDM','ARNE','BBOW',
              'BEAV','BEEX','BESS','BIXB','BLAC','BOIS','BOWL','BREC','BRIS','BROK','BUFF','BURB',
              'BURN','BUTL','BYAR','CALV','CAMA','CARL','CATO','CENT','CHAN','CHER','CHEY','CHIC','CLAR','CLAY',
              'CLOU','CLRM','COOK','COPA','DURA','ELKC','ELRE','ERIC','EUFA','EVAX','FAIR','FITT','FORA','FREE',
              'FTCB','GOOD','GRA2','GRAN','GUTH','HASK','HECT','HINT','HOBA','HOLD','HOLL','HOOK','HUGO','IDAB','INOL',
              'JAYX','KENT','KETC','KIN2','KING','LAHO','LANE','MADI','MANG','MARE','MARS','MAYR','MCAL','MEDF','MEDI',
              'MIAM','MINC','MRSH','MTHE','NEWK','NEWP','NINN','NORM','NOWA','NRMN','OILT','OKCE','OKCN','OKCW','OKEM',
              'OKMU','PAUL','PAWN','PERK','PORT','PRES','PRYO','PUTN','REDR','RETR','RING','SALL','SEIL','SEMI','SHAW',
              'SKIA','SLAP','SPEN','STIG','STIL','STUA','SULP','TAHL','TALA','TALI','TIPT','TISH','TULL','TULN','VALL',
              'VANO','VINI','WAL2','WALT','WASH','WATO','WAUR','WEAT','WEB3','WEBB','WEBR','WEST','WILB','WIST','WOOD',
              'WYNO','YUKO']
    print("Reading in Mesonet data....")
    
    
    # Start and End dates and times for loop
    period_start = datetime(2014,9,1,0,0)
    period_end = datetime(2014,9,30,23,55)
    
    # Create blank final data ratio dataframes
    final_ratio_70_090=pd.DataFrame()
#    final_ratio_70_095=pd.DataFrame()
#    final_ratio_70_100=pd.DataFrame()
#    final_ratio_70_105=pd.DataFrame()
#    final_ratio_70_110=pd.DataFrame()
#    final_ratio_70_115=pd.DataFrame()
#    final_ratio_70_120=pd.DataFrame()
    
    final_ratio_75_090=pd.DataFrame()
#     final_ratio_75_095=pd.DataFrame()
#     final_ratio_75_100=pd.DataFrame()
#     final_ratio_75_105=pd.DataFrame()
#     final_ratio_75_110=pd.DataFrame()
#     final_ratio_75_115=pd.DataFrame()
#     final_ratio_75_120=pd.DataFrame()
    
#    final_ratio_80_090=pd.DataFrame()
#     final_ratio_80_095=pd.DataFrame()
#     final_ratio_80_100=pd.DataFrame()
#     final_ratio_80_105=pd.DataFrame()
#     final_ratio_80_110=pd.DataFrame()
#     final_ratio_80_115=pd.DataFrame()
#     final_ratio_80_120=pd.DataFrame()
    
    
    #Loop through months
    print('Starting Loop')
    curr_date = period_start
    while curr_date <= period_end:
        print("{} ".format(curr_date.month), end="")
        month_start= datetime(curr_date.year,curr_date.month,1,0,0)
        month_end= datetime(curr_date.year,curr_date.month,calendar.monthrange(curr_date.year,curr_date.month)[1],23,55)
        
       
        
        #Create data frame for 70% dewpoint depression ratio over 18 to 24 5 minute intervals (90-120 minutes)
        ratio_70_090=pd.DataFrame()
#       ratio_70_095=pd.DataFrame()
#       ratio_70_100=pd.DataFrame()
#       ratio_70_105=pd.DataFrame()
#       ratio_70_110=pd.DataFrame()
#       ratio_70_115=pd.DataFrame()
#       ratio_70_120=pd.DataFrame()
  
        #Create data frame for 75% dewpoint depression ratio over 18 to 24 5 minute intervals (90-120 minutes)
        ratio_75_090=pd.DataFrame()
#       ratio_75_095=pd.DataFrame()
#       ratio_75_100=pd.DataFrame()
#       ratio_75_105=pd.DataFrame()
#       ratio_75_110=pd.DataFrame()
#       ratio_75_115=pd.DataFrame()
#       ratio_75_120=pd.DataFrame()

        #Create data frame for 80% dewpoint depression ratio over 18 to 24 5 minute intervals (90-120 minutes)
#       ratio_80_090=pd.DataFrame()
#       ratio_80_095=pd.DataFrame()
#       ratio_80_100=pd.DataFrame()
#       ratio_80_105=pd.DataFrame()
#       ratio_80_110=pd.DataFrame()
#       ratio_80_115=pd.DataFrame()
#       ratio_80_120=pd.DataFrame()

        #Bring in parameters
        mesonet_data = mesonet_time_series_data(stids, month_start, month_end, params)
        mesonet_data = mesonet_data.fillna(value=np.nan)
        mesonet_data['date']=mesonet_data['time'].dt.date
        

        #Calculate max wind gust
        mesonet_data['max_gust']=mesonet_data['wmax'].rolling(5).max().shift(-3)

        #Calculate initialdew point depression
        mesonet_data['dewd']= mesonet_data['tair']-mesonet_data['tdew']
        
        #Create new dataframe to check if station that is 2 ahead matches station
        mesonet_data['matches?'] = mesonet_data['stid'].shift(-2)==mesonet_data['stid']
            
        #Calculate dew point depression change over various intervals and add to dataframe
        mesonet_data['dewd_090_change']=mesonet_data['dewd'].shift(18)- mesonet_data['dewd']
#       mesonet_data['dewd_095_change']=mesonet_data['dewd'].shift(19)- mesonet_data['dewd']
#       mesonet_data['dewd_100_change']=mesonet_data['dewd'].shift(20)- mesonet_data['dewd']
#       mesonet_data['dewd_105_change']=mesonet_data['dewd'].shift(21)- mesonet_data['dewd']
#       mesonet_data['dewd_110_change']=mesonet_data['dewd'].shift(22)- mesonet_data['dewd']
#       mesonet_data['dewd_115_change']=mesonet_data['dewd'].shift(23)- mesonet_data['dewd']
#       mesonet_data['dewd_120_change']=mesonet_data['dewd'].shift(24)- mesonet_data['dewd']
        
        #Calculate 10 minute air temperature and dew point changes and add to dataframe
        mesonet_data['tair_10min_change']= mesonet_data['tair'].shift(-2)-mesonet_data['tair']
        mesonet_data['dewd_10min_change']= (mesonet_data['dewd'].shift(-2))-(mesonet_data['dewd'])
                                            
        #Calculation for T_Δt is the dry-bulb temperature after some change in time
        mesonet_data['tair_diff_090']=mesonet_data['tair'].shift(18)- mesonet_data['tair']
#       mesonet_data['tair_diff_095']=mesonet_data['tair'].shift(19)- mesonet_data['tair']
#       mesonet_data['tair_diff_100']=mesonet_data['tair'].shift(20)- mesonet_data['tair']
#       mesonet_data['tair_diff_105']=mesonet_data['tair'].shift(21)- mesonet_data['tair']
#       mesonet_data['tair_diff_110']=mesonet_data['tair'].shift(22)- mesonet_data['tair']
#       mesonet_data['tair_diff_115']=mesonet_data['tair'].shift(23)- mesonet_data['tair']
#       mesonet_data['tair_diff_120']=mesonet_data['tair'].shift(24)- mesonet_data['tair']
        
                          
        #Calculation for T_dΔt is the dew point temperature after some change in time
        mesonet_data['tdew_diff_090']=mesonet_data['tdew'].shift(18)-mesonet_data['tdew']
#       mesonet_data['tdew_diff_095']=mesonet_data['tdew'].shift(19)-mesonet_data['tdew']
#       mesonet_data['tdew_diff_100']=mesonet_data['tdew'].shift(20)-mesonet_data['tdew']
#       mesonet_data['tdew_diff_105']=mesonet_data['tdew'].shift(21)-mesonet_data['tdew']
#       mesonet_data['tdew_diff_110']=mesonet_data['tdew'].shift(22)-mesonet_data['tdew']
#       mesonet_data['tdew_diff_115']=mesonet_data['tdew'].shift(23)-mesonet_data['tdew']
#        mesonet_data['tdew_diff_120']=mesonet_data['tdew'].shift(24)-mesonet_data['tdew']

        #Calculate dew point depresssion ratio
        mesonet_data['tair2']=mesonet_data['tair'].shift(-18)
        mesonet_data['tdew2']=mesonet_data['tdew'].shift(-18)
        mesonet_data['dewpt_dpr_ratio_090']=((mesonet_data['tair']-(mesonet_data['tdew']))/(mesonet_data['tair'].shift(18)-(mesonet_data['tdew'].shift(18))))
#       mesonet_data['dewpt_dpr_ratio_095']=((mesonet_data['tair']-mesonet_data['tdew'])/(mesonet_data['tair'].shift(19)-mesonet_data['tdew'].shift(19)))
#       mesonet_data['dewpt_dpr_ratio_100']=((mesonet_data['tair']-mesonet_data['tdew'])/(mesonet_data['tair'].shift(20)-mesonet_data['tdew'].shift(20)))
#       mesonet_data['dewpt_dpr_ratio_105']=((mesonet_data['tair']-mesonet_data['tdew'])/(mesonet_data['tair'].shift(21)-mesonet_data['tdew'].shift(21)))
#       mesonet_data['dewpt_dpr_ratio_110']=((mesonet_data['tair']-mesonet_data['tdew'])/(mesonet_data['tair'].shift(22)-mesonet_data['tdew'].shift(22)))
#       mesonet_data['dewpt_dpr_ratio_115']=((mesonet_data['tair']-mesonet_data['tdew'])/(mesonet_data['tair'].shift(23)-mesonet_data['tdew'].shift(23)))
#       mesonet_data['dewpt_dpr_ratio_120']=((mesonet_data['tair']-mesonet_data['tdew'])/(mesonet_data['tair'].shift(24)-mesonet_data['tdew'].shift(24)))
        
        
        #Calculate pre and post dew point depression and find change to use for criteria
        mesonet_data['Pre_dewpt_dpr']=(mesonet_data['tair'].shift(-6)-(mesonet_data['tdew'].shift(-6)))
        mesonet_data['Post_dewpt_dpr']=(mesonet_data['tair'].shift(60)-mesonet_data['tdew'].shift(60))
        mesonet_data['Dewpt_dpr_change']=abs(mesonet_data['Post_dewpt_dpr']-mesonet_data['Pre_dewpt_dpr'])

    
        #Add to data frame for 70% dewpoint depression ratio and above
        ratio_70_090= ratio_70_090.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_090']>=0.70)], ignore_index=True)
#       ratio_70_095= ratio_70_095.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_095']>=0.70)], ignore_index=True)
#       ratio_70_100= ratio_70_100.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_100']>=0.70)], ignore_index=True)
#       ratio_70_105= ratio_70_105.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_105']>=0.70)], ignore_index=True)
#       ratio_70_110= ratio_70_110.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_110']>=0.70)], ignore_index=True)
#       ratio_70_115= ratio_70_115.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_115']>=0.70)], ignore_index=True)
#       ratio_70_120= ratio_70_120.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_120']>=0.70)], ignore_index=True)

        #Add to data frame for 75% dewpoint depression ratio and above
#       ratio_75_090= ratio_75_090.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_090']>=0.75)], ignore_index=True)
#       ratio_75_095= ratio_75_095.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_095']>=0.75)], ignore_index=True)
#       ratio_75_100= ratio_75_100.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_100']>=0.75)], ignore_index=True)
#       ratio_75_105= ratio_75_105.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_105']>=0.75)], ignore_index=True)
#       ratio_75_110= ratio_75_110.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_110']>=0.75)], ignore_index=True)
#       ratio_75_115= ratio_75_115.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_115']>=0.75)], ignore_index=True)
#       ratio_75_120= ratio_75_120.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_120']>=0.75)], ignore_index=True)

        #Add to data frame for 80% dewpoint depression ratio and above
#       ratio_80_090= ratio_80_090.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_090']>=0.80)], ignore_index=True)
#       ratio_80_095= ratio_80_095.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_095']>=0.80)], ignore_index=True)
#       ratio_80_100= ratio_80_100.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_100']>=0.80)], ignore_index=True)
#       ratio_80_105= ratio_80_105.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_105']>=0.80)], ignore_index=True)
#       ratio_80_110= ratio_80_110.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_110']>=0.80)], ignore_index=True)
#       ratio_80_115= ratio_80_115.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_115']>=0.80)], ignore_index=True)
#       ratio_80_120= ratio_80_120.append(mesonet_data[(mesonet_data['dewpt_dpr_ratio_120']>=0.80)], ignore_index=True)


        #Criteria:
        #1. Tair 10 min change is greater than or equal to tair threshold
        #2. Dew point depression 10 minute chage is greater than or equal to tdew deppression threshold
        #3. Wind gusts is greater than or equal to wind gusts threshold
        #4. (Dewd/dewd_(insert time)_change) greater than or equal to (ratio threshold/100)
        #5. Station that is 2 ahead matches station
        #6. Dew Point Depression has returned to less than or equal dew point depression threshold


        ratio_70_090['heat burst']=ratio_70_090.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
                                                         (row['dewd_10min_change']>= tdew_depr_threshold) and
                                                         (row['max_gust']>= gust_max_threshold) and
                                                         (row['matches?']== True))                
                                                         #(row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
                                                         else 0, axis=1)
    
#         ratio_70_095['heat burst']=ratio_70_095.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                        (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                        (row['max_gust']>= gust_max_threshold) and
#                                                        (row['matches?']== True) and                
#                                                        (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                        else 0, axis=1)
    
#         ratio_70_100['heat burst']=ratio_70_100.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_70_105['heat burst']=ratio_70_105.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_70_110['heat burst']=ratio_70_110.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_70_115['heat burst']=ratio_70_115.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_70_120['heat burst']=ratio_70_120.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_75_090['heat burst']=ratio_75_090.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_75_095['heat burst']=ratio_75_095.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_75_100['heat burst']=ratio_75_100.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)
                    
#         ratio_75_105['heat burst']=ratio_75_105.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)
                            
#         ratio_75_110['heat burst']=ratio_75_110.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_75_115['heat burst']=ratio_75_115.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_75_120['heat burst']=ratio_75_120.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_80_090['heat burst']=ratio_80_090.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                        (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                        (row['max_gust']>= gust_max_threshold) and
#                                                        (row['matches?']== True) and                
#                                                        (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                        else 0, axis=1)

#         ratio_80_095['heat burst']=ratio_80_095.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_80_100['heat burst']=ratio_80_100.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)
                            
#         ratio_80_105['heat burst']=ratio_80_105.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_80_110['heat burst']=ratio_80_110.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)


#         ratio_80_115['heat burst']=ratio_80_115.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

#         ratio_80_120['heat burst']=ratio_80_120.apply(lambda row: 1 if ((row['tair_10min_change']>=tair_threshold) and 
#                                                       (row['dewd_10min_change']>= tdew_depr_threshold) and
#                                                       (row['max_gust']>= gust_max_threshold) and
#                                                       (row['matches?']== True) and                
#                                                       (row['Dewpt_dpr_change']<= dewpt_dpt_change_threshold))                
#                                                       else 0, axis=1)

        #Append to final ratio dataframe
        final_ratio_70_090 = final_ratio_70_090.append(ratio_70_090[ratio_70_090['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_70_095 = final_ratio_70_095.append(ratio_70_095[ratio_70_095['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_70_100 = final_ratio_70_100.append(ratio_70_100[ratio_70_100['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_70_105 = final_ratio_70_105.append(ratio_70_105[ratio_70_105['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_70_110 = final_ratio_70_110.append(ratio_70_110[ratio_70_110['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_70_115 = final_ratio_70_115.append(ratio_70_115[ratio_70_115['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_70_120 = final_ratio_70_120.append(ratio_70_120[ratio_70_120['heat burst']>0].groupby(['date','stid']).first())
        
#        final_ratio_75_090 = final_ratio_75_090.append(ratio_75_090[ratio_75_090['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_75_095 = final_ratio_75_095.append(ratio_75_095[ratio_75_095['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_75_100 = final_ratio_75_100.append(ratio_75_100[ratio_75_100['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_75_105 = final_ratio_75_105.append(ratio_75_105[ratio_75_105['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_75_110 = final_ratio_75_110.append(ratio_75_110[ratio_75_110['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_75_115 = final_ratio_75_115.append(ratio_75_115[ratio_75_115['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_75_120 = final_ratio_75_120.append(ratio_75_120[ratio_75_120['heat burst']>0].groupby(['date','stid']).first())
        
#        final_ratio_80_090 = final_ratio_80_090.append(ratio_80_090[ratio_80_090['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_80_095 = final_ratio_80_095.append(ratio_80_095[ratio_80_090['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_80_100 = final_ratio_80_100.append(ratio_80_100[ratio_80_100['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_80_105 = final_ratio_80_105.append(ratio_80_105[ratio_80_105['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_80_110 = final_ratio_80_110.append(ratio_80_110[ratio_80_110['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_80_115 = final_ratio_80_115.append(ratio_80_115[ratio_80_115['heat burst']>0].groupby(['date','stid']).first())
#        final_ratio_80_120 = final_ratio_80_120.append(ratio_80_120[ratio_80_120['heat burst']>0].groupby(['date','stid']).first())
        
        
        #Increment month
        curr_date = curr_date + relativedelta(months=1)
    
    ####End of loop####
    print('Ending loop')
    
    #Save dataframes to CSV
    final_ratio_70_090.to_csv('{}_Sept_Ratio_70_090.csv'.format(period_start.year))
#    final_ratio_70_095.to_csv('{}_Ratio_70_095.csv'.format(period_start.year))
#    final_ratio_70_100.to_csv('{}_Ratio_70_100.csv'.format(period_start.year))
#    final_ratio_70_105.to_csv('{}_Ratio_70_105.csv'.format(period_start.year))
#    final_ratio_70_110.to_csv('{}_Ratio_70_110.csv'.format(period_start.year))
#    final_ratio_70_115.to_csv('{}_Ratio_70_115.csv'.format(period_start.year))
#    final_ratio_70_120.to_csv('{}_Ratio_70_120.csv'.format(period_start.year))
    
#    final_ratio_75_090.to_csv('{}_Ratio_75_090.csv'.format(period_start.year))
#    final_ratio_75_095.to_csv('{}_Ratio_75_095.csv'.format(period_start.year))
#    final_ratio_75_100.to_csv('{}_Ratio_75_100.csv'.format(period_start.year))
#    final_ratio_75_105.to_csv('{}_Ratio_75_105.csv'.format(period_start.year))
#    final_ratio_75_110.to_csv('{}_Ratio_75_110.csv'.format(period_start.year))
#    final_ratio_75_115.to_csv('{}_Ratio_75_115.csv'.format(period_start.year))
#    final_ratio_75_120.to_csv('{}_Ratio_75_120.csv'.format(period_start.year))
    
#    final_ratio_80_090.to_csv('{}_Ratio_80_090.csv'.format(period_start.year))
#    final_ratio_80_095.to_csv('{}_Ratio_80_095.csv'.format(period_start.year))
#    final_ratio_80_100.to_csv('{}_Ratio_80_100.csv'.format(period_start.year))
#    final_ratio_80_105.to_csv('{}_Ratio_80_105.csv'.format(period_start.year))
#    final_ratio_80_110.to_csv('{}_Ratio_80_110.csv'.format(period_start.year))
#    final_ratio_80_115.to_csv('{}_Ratio_80_115.csv'.format(period_start.year))
#    final_ratio_80_120.to_csv('{}_Ratio_80_120.csv'.format(period_start.year))
    
    print("** End Program **")
    



** Begin Program **
Reading in Mesonet data....
Starting Loop
9 Ending loop
** End Program **


In [ ]:
ratio_70_090[580630:580690][['matches?','stid','tair','time','tair_10min_change','dewd_10min_change',"max_gust", 'tdew','Dewpt_dpr_change']]

In [ ]:
mesonet_data[30:][['stid','tair','time','tair2','tdew2','tair_10min_change','dewd_10min_change','wmax',"max_gust", 'tdew','dewpt_dpr_ratio_090']]

In [ ]:
ratio_70_090[ratio_70_090['heat burst']>0][['stid','time','tair_10min_change','dewd_10min_change','wmax',"max_gust", 'tdew']]

In [ ]:
temp=ratio_70_090[ratio_70_090['heat burst']>0][['stid','time','tair_10min_change','dewd_10min_change','wmax',"max_gust", 'tdew']]
temp=temp.reset_index(drop=False)
temp.groupby(['date']).first()

In [ ]:
temp

In [ ]:
mesonet_data['tair']

In [ ]:
#Caluclate Wind Gust
df = pd.DataFrame({'wmax': [23,15,92,75,110,112,78,56,38,23,17,10,1]})
df['max_gust']=df['wmax'].rolling(5).max().shift(-3)
df